## Types of Timers

- [Basic](#Basic-Timers)
- [General Purpose](#General-Purpose-Timers)
- Advanced
- High Resolution
- Low Power



### Basic Timers
- time-based generation
- drive ADC/DAC (internally connected to those peripherals)

### Time Frequency
Some important registers:

Register | Description
---|---
 Counter (CNT) | The current counter value 
 Auto-Reload Register (ARR) | The value for the timer to count up to 
 Pre-scaler (PSC) | Clock Divider frequency 




The timer frequency, `Update Event`, is determined by the period (`ARR`) and prescaler (`PSC`):

`Timer Frequency (Hz) = Timer Clock (Hz) / (( PSC + 1) (ARR + 1))`


In [19]:
timer_clock_hz = 4000000 # 4 MHz clock
psc = 63
arr = 62499
timer_frequency = timer_clock_hz / ((psc + 1) * (arr + 1))
print("Update Frequency: " + str(timer_frequency) + " Hz or " + str(1/timer_frequency) + " s")

Update Frequency: 1.0 Hz or 1.0 s


### Resolution & Granularity
Resolution is the maximum wait period of the timer. 
For example, on an 8-bit timer which a clock frequency of 1 MHz (1 cycle = 1 us), the resolution of the timer is 256 us.
```Python
Resolution = 2^n * timer period, where n is the bit-resolution of the timer
```

In [4]:
timer_bits = 8
timer_freq = 1000000
resolution = (2**timer_bits)*(1/timer_freq)
print(str(resolution) + " s")

0.000256 s


The resolution can be lowered by via prescaler/auto reload/repetition counter.

When lowering the resolution by using the prescaler, the **granularity** of also increases.
For example, on an 8-bit timer with 1 MHz timer clock:
- no presclaer: 1 timer tick = 1 clock cycle -> 1 us
- 1024 presclaer: 1 timer tick = 1024 cycles -> 1.024 ms

### General Purpose Timers
General purpose timers have independent channels to:
- measure input signals
- output signals on time basis
- generate PWM

Some general purpose timers also introduce the concept of 'Repetition Counter'. When the repetition counter counts down to zero, the update event flag is raised. Technically, basic timers also have a repetitition counter, however it is always set to 1.

``` C++
                                Timer Clock (Hz) 
Timer Frequency (Hz) = --------------------------------
                        ( PSC + 1) (ARR + 1) (RCR + 1)
```

![General Purpose Timer Block Diagram](resources/general_purpose_timer_block_diagram.PNG)

#### Input Capture
*https://www.youtube.com/watch?v=2FoZ7kHOdT0*
- captures the timestamp of an external events.
    - events are characterized by rising and/or falling edge of a digital signal
    - events are detected through the `Input filter & edge detector` block

Simple Example:
Suppose `ARR=99` and events are captured on rising edges.
![Simple Input Capture Example](resources/simple_input_capture_example.png)

Time unit is the period of the counter clock.



##### How to setup:
1. Enable the appropriate timer clock and GPIO clock in the RCC
2. Configure the appropriate GPIO pin for the Timer alternate function
3. Map a channel to a timer input by writing the appropriate CCxS bits to the CCMRx register.
    - For example, to configure Channel 3 as input channel on TI3, we would write the bits `0b01` to CC3S in TIM2->CCMR2
4. Configure the active edge of transition by writing to the CCxP/CCxNP bits in the CCER register
    - For example, to configure channel 3 to detect rising edges: `CC3P = 0` and `CC3NP = 0`
5. Optionally, enable the capture/compare interrupts if required
6. Enable the capture/compare module by setting the `CCxE` bit in the `CCER` register
7. Enable the counter module by setting the `CEN` bit in the `CR1` register

# Other Resources
https://ti.tuwien.ac.at/ecs/teaching/courses/mcvo-ss08/files/VO-07-CounterTimer.pdf